#10가지 분류 알고리즘, 정확도/AUC/F1-score 평가

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

%cd /content/gdrive/My Drive/Colab Notebooks/BAMT-GAN/evaluation/

Mounted at /content/gdrive/
/content/gdrive/My Drive/Colab Notebooks/BAMT-GAN/evaluation


##import

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.3 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

import pandas as pd
from catboost import CatBoostClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib as mlp

%matplotlib inline

from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# import machine learning algorithms
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


##성능 평가

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encoding_df(df):
  df = df.apply(LabelEncoder().fit_transform)
  return df

In [ ]:
def classification_eval(df, target):
    df = label_encoding_df(pd.read_csv(df))
    X = df.drop(labels=target, axis=1)
    y = df[target]

    # Add additional models
    models = [
        ('DT', tree.DecisionTreeClassifier()),
        ('KNN', KNeighborsClassifier()),
        ('LR', make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))),
        ('RF', RandomForestClassifier()),
       ('GBM', GradientBoostingClassifier()),
        ('XGBoost', XGBClassifier(eval_metric='mlogloss')),
        ('LightGBM', LGBMClassifier()),
        ('CatBoost', CatBoostClassifier(verbose=0)),
        ('LinearSVM', make_pipeline(StandardScaler(), LinearSVC())),
        ('AdaBoost', AdaBoostClassifier())
    ]


    scoring = {'accuracy': make_scorer(accuracy_score),
               'auc': make_scorer(roc_auc_score),
               'f1': make_scorer(f1_score)}

    result_dict = {}
    for name, model in models:
        print(name)
        kfold = KFold(n_splits=5, shuffle=True, random_state=10)
        cv_results = cross_validate(model, X, y, cv=kfold, scoring=scoring)
        print('Accuracy : mean {0:.2f}, std {1:.2f} | AUC : mean {2:.2f}, std {3:.2f} | F1-Score : mean {4:.2f}, std {5:.2f}'.format(
            cv_results['test_accuracy'].mean()*100, cv_results['test_accuracy'].std()*100,
            cv_results['test_auc'].mean()*100, cv_results['test_auc'].std()*100,
            cv_results['test_f1'].mean()*100, cv_results['test_f1'].std()*100))
        result_dict[name] = [cv_results['test_accuracy'].mean()*100, cv_results['test_accuracy'].std()*100,
                              cv_results['test_auc'].mean()*100, cv_results['test_auc'].std()*100,
                              cv_results['test_f1'].mean()*100, cv_results['test_f1'].std()*100]

    return result_dict


In [13]:
from sklearn.impute import SimpleImputer

def ratio_eval(df, df_fake, target,name):

    imputer = SimpleImputer(strategy='mean')
    result_df = pd.DataFrame()
    orig_result_dict = new_ml_eval2(df, target)

    # Save the performance of the original data
    for key, value in orig_result_dict.items():
        result_df.loc['Original', key+'_accuracy_mean'] = value[0]
        result_df.loc['Original', key+'_accuracy_std'] = value[1]
        result_df.loc['Original', key+'_auc_mean'] = value[2]
        result_df.loc['Original', key+'_auc_std'] = value[3]
        result_df.loc['Original', key+'_f1_mean'] = value[4]
        result_df.loc['Original', key+'_f1_std'] = value[5]

    # Save the performance of the augmented data
    augmentation_ratio = [0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0]
    for i in augmentation_ratio:
        print(str(i*100)+'%------------------')
        aug_ratio_df = new_balanced_sampling_and_augmentation(df, df_fake, i, target)
        aug_ratio_df.to_csv('./new_augmented_data/'+str(i)+name+'_bamtgan.csv',index=False)
        aug_ratio_df[aug_ratio_df.columns] = imputer.fit_transform(aug_ratio_df)
        result_dict = new_ml_eval2(aug_ratio_df,target)

        for key, value in result_dict.items():
            result_df.loc[str(i*100)+'%', key+'_accuracy_mean'] = value[0]
            result_df.loc[str(i*100)+'%', key+'_accuracy_std'] = value[1]
            result_df.loc[str(i*100)+'%', key+'_auc_mean'] = value[2]
            result_df.loc[str(i*100)+'%', key+'_auc_std'] = value[3]
            result_df.loc[str(i*100)+'%', key+'_f1_mean'] = value[4]
            result_df.loc[str(i*100)+'%', key+'_f1_std'] = value[5]

        aug_ratio_df[target].value_counts()

    # Add augmentation ratio as a separate column at the beginning
    result_df.reset_index(inplace=True)
    result_df.rename(columns={'index': 'Augmentation Ratio'}, inplace=True)
    result_df.set_index('Augmentation Ratio', inplace=True)

    return result_df

##함수 호출

In [ ]:
tox21_raw =pd.read_csv('../data/tox21.csv')
tox21_fake = pd.read_csv('./')
tox21 = ratio_eval(tox21_raw,tox21_fake,'SR.ARE','tox21')
data = classification_eval('../augmented_data/tox21/0.2tox21_bamtgan.csv', 'SR.ARE')

DT
Accuracy : mean 87.08, std 0.63 | AUC : mean 86.80, std 0.75 | F1-Score : mean 84.52, std 1.06
KNN
Accuracy : mean 89.48, std 0.39 | AUC : mean 88.11, std 0.52 | F1-Score : mean 86.33, std 0.72
LR
Accuracy : mean 89.01, std 0.40 | AUC : mean 87.28, std 0.55 | F1-Score : mean 85.34, std 0.76
RF
Accuracy : mean 91.05, std 0.33 | AUC : mean 89.44, std 0.42 | F1-Score : mean 88.12, std 0.53
GBM
Accuracy : mean 90.17, std 0.30 | AUC : mean 88.47, std 0.59 | F1-Score : mean 86.87, std 0.78
XGBoost
Accuracy : mean 91.18, std 0.51 | AUC : mean 89.97, std 0.65 | F1-Score : mean 88.62, std 0.80
LightGBM
Accuracy : mean 91.11, std 0.48 | AUC : mean 89.76, std 0.65 | F1-Score : mean 88.41, std 0.81
CatBoost
Accuracy : mean 91.19, std 0.29 | AUC : mean 89.71, std 0.47 | F1-Score : mean 88.41, std 0.60
LinearSVM


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy : mean 89.17, std 0.37 | AUC : mean 87.19, std 0.70 | F1-Score : mean 85.26, std 0.94
AdaBoost
Accuracy : mean 89.60, std 0.46 | AUC : mean 88.04, std 0.73 | F1-Score : mean 86.27, std 0.97
